# synthesizable contexts

CoHDL translates entities, by executing the architecture method as normal Python code and collecting all function definitions marked as synthesizable contexts. After returning from `architecture` CoHDL parses all found contexts and generates equivalent VHDL. There are two types of synthesizable contexts:

* concurrent contexts:
    - are decorated with `std.concurrent` (or `concurrent_context`)
    - are translated into statements outside VHDL processes
* sequential contexts:
    - are decorated with `std.sequential` (or `sequential_context`)
    - are translated into VHDL processes

Footnote:

`std.concurrent` and `std.sequential` are convenience wrappers and should be preferred over the underlying CoHDL builtins `concurrent_context` and `sequential_context`.

---
## concurrent logic

The `std.concurrent` marks code that should not be part of a VHDL process. To translate the inverter example from the previous section, CoHDL performs the following steps:

1. the compiler calls `architecture` as a normal Python function
2. the `std.concurrent` decorator adds the local function `logic` to a global list of all synthesizable contexts
3. once `architecture` returns the compiler translates all entries in that list to an internal representation
4. the VHDL compiler backend takes this representation and turns it into VHDL (currently the only supported target language)
5. this VHDL code is placed directly in the architecture scope (not in a process)

In [1]:
from cohdl import Entity, Port, Bit
from cohdl import std

class Inverter(Entity):
    input = Port.input(Bit)
    output = Port.output(Bit)

    def architecture(self):
        @std.concurrent
        def logic():
            self.output <<= ~self.input

print(std.VhdlCompiler.to_string(Inverter))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity Inverter is
  port (
    input : in std_logic;
    output : out std_logic
    );
end Inverter;


architecture arch_Inverter of Inverter is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_output : std_logic;
  signal temp : std_logic;
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  output <= buffer_output;
  
  -- CONCURRENT BLOCK (logic)
  temp <= not (input);
  buffer_output <= temp;
end architecture arch_Inverter;


---
## sequential logic

The `std.sequential` decorator works like `std.concurrent`. However the generated VHDL representation is placed in a process. The following examples show three different ways of implementing a counter that is incremented on every rising edge of a clock signal.

### `rising_edge`

CoHDL defines `rising_edge` and `falling_edge` as a magic functions, to make it possible to implement VHDL style clocked logic.

In [2]:
from cohdl import Entity, Port, Bit, Unsigned, rising_edge
from cohdl import std

class Counter(Entity):
    clk = Port.input(Bit)

    output = Port.output(Unsigned[16])

    def architecture(self):
        @std.sequential
        def process():
            if rising_edge(self.clk):
                # reading from output ports is supported
                # cohdl introduces an additional intermediary
                # signal for that purpose
                self.output <<= self.output + 1

print(std.VhdlCompiler.to_string(Counter))

'_BitSignalEvent' object has no attribute '__bool__'
    @ /tmp/ipykernel_253511/3177613936.py:12

used in <class 'ast.Call'>
    @ /home/alexander/dev/github/cohdl/cohdl/std/_context.py:327

used in <class 'ast.Expr'>
    @ /home/alexander/dev/github/cohdl/cohdl/std/_context.py:327

used in <class 'list'>
    @ /home/alexander/dev/github/cohdl/cohdl/std/_context.py:320

used in <class 'ast.If'>
    @ /home/alexander/dev/github/cohdl/cohdl/std/_context.py:324



AttributeError: '_BitSignalEvent' object has no attribute '__bool__'

### `std.Clock`

In addition `std.sequential` also provides a [MyHDL](https://github.com/myhdl/myhdl) style interface for clocked logic and resets. It is based on the types `std.Clock` and `std.Reset` both of which wrap bit signals.

When a `std.Clock` parameter is given to `std.sequential`, the HDL representation is wrapped in an if statement. Clocks can be sensitive to rising, falling or both edges. Additional parameters like frequency or duty cycle can be specified, in the constructor of the `Clock` object.

In [ ]:
from cohdl import Entity, Port, Bit, Unsigned
from cohdl import std

class Counter(Entity):
    clk = Port.input(Bit)
    output = Port.output(Unsigned[16])

    def architecture(self):
        clk = std.Clock(self.clk)

        # passing a clock parameter to std.sequential
        # wraps the entire function body in an
        # if rising_edge(clk_signal) statement
        @std.sequential(clk)
        def process():
            self.output <<= self.output + 1

print(std.VhdlCompiler.to_string(Counter))

### `std.Reset`

When a `std.Reset` parameter is given to `std.sequential`, the HDL representation is wrapped in an additional if statement. Depending on the state of the reset signal, it either executes the function body or resets all Signals and Variables, that are driven by the this context and have default values.

`std.Reset` can be synchronous or asynchronous and high or low active. `std.sequential` generates the order of the if-clock_edge-statement and the if-reset-statement accordingly.

In [ ]:
from cohdl import Entity, Port, Bit, Unsigned
from cohdl import std

class Counter(Entity):
    clk = Port.input(Bit)
    reset = Port.input(Bit)

    output = Port.output(Unsigned[16], default=0)

    def architecture(self):
        clk = std.Clock(self.clk)

        # By default std.Reset is active high and synchronous
        # it can also be configured as active low and/or asynchronous.
        reset = std.Reset(self.reset)

        @std.sequential(clk, reset)
        def process():
            self.output <<= self.output + 1

print(std.VhdlCompiler.to_string(Counter))